In [ ]:
import speech_recognition as sr
recognizer = sr.Recognizer()

from transformers import pipeline
summarizer = pipeline("summarization")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import speech_recognition as sr
from gtts import gTTS
import pygame
import io
import time




def speak(text):
    """Speak text using Google Text-to-Speech"""
    if not text:
        return

    try:
        # Initialize pygame mixer if not already initialized
        if not pygame.mixer.get_init():
            pygame.mixer.init()

        # Create gTTS object and save to BytesIO
        tts = gTTS(text=text, lang='en', slow=False)
        fp = io.BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)

        # Play the audio
        pygame.mixer.music.load(fp)
        pygame.mixer.music.play()

        # Wait for audio to finish
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    except Exception as e:
        print(f"Error in speech: {e}")

def get_voice_input():
    """Get voice input from user and convert to text"""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Please speak the company name...")
        speak("Please speak the company name")
        recognizer.adjust_for_ambient_noise(source, duration=3)
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            speak(f"You said {text}")
            return text.upper()
        except Exception as e:
            print(f"Error in voice recognition: {e}")
            speak("I couldn't understand that. Please try again")
            return None

def extract_ticker(spoken_text):
    """Extract ticker symbol from spoken text using S&P 500 companies list"""
    if not spoken_text:
        return None

    sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables_sp500 = pd.read_html(sp500_url)
    sp500_tickers = tables_sp500[0]['Symbol'].tolist()

    # Get NASDAQ tickers
    nasdaq_url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    tables_nasdaq = pd.read_html(nasdaq_url)
    # The NASDAQ table might have a different structure, adjust the column name if needed
    nasdaq_tickers = tables_nasdaq[4]['Symbol'].tolist()  # Index and column name might need adjustment

    # Combine both ticker lists
    known_tickers = list(set(sp500_tickers + nasdaq_tickers))

    words = spoken_text.split()
    for word in words:
        if word in known_tickers:
            return word
    return None

def get_news(ticker):
    """Fetch news using Alpha Vantage API"""
    api_key = '---------' #API Key
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&limit=3&apikey={api_key}'

    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def get_article_text(url):
    """Extract text from article URL"""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all(['h1', 'p'])
        return ' '.join([p.get_text() for p in paragraphs])
    except Exception as e:
        print(f"Error fetching article: {e}")
        return None

def summarize_text(text, summarizer):
    """Summarize text using the provided summarizer"""
    if not text:
        return "Unable to summarize article."

    ARTICLE = text.replace('.', '.<eos>').replace('!', '!<eos>').replace('?', '?<eos>')
    sentences = ARTICLE.split('<eos>')

    max_chunk = 500
    chunks = []
    current_chunk = 0
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    summaries = []
    for chunk in chunks:
        input_length = len(chunk.split())
        max_length = min(120, max(30, input_length))
        summary = summarizer(chunk, max_length=max_length, min_length=min(30, input_length), do_sample=False)
        summaries.extend(summary)

    return ' '.join([summ['summary_text'] for summ in summaries])

def format_news(news_data, summarizer):
    """Format news data with summaries and read them aloud"""
    if not news_data or 'feed' not in news_data:
        msg = "No news available."
        speak(msg)
        return msg

    formatted_output = []
    for i, article in enumerate(news_data['feed'][:3], 1):
        article_text = get_article_text(article['url'])
        if article_text:
            summary = summarize_text(article_text, summarizer)
            article_content = (
                f"Article {i}:\n"
                f"Title: {article['title']}\n"
                f"Source: {article['source']}\n"
                f"Summary: {summary}\n"
            )
            formatted_output.append(article_content)

            # Read each article
            speak(f"Article {i}")
            time.sleep(0.5)
            speak(f"Title: {article['title']}")
            time.sleep(0.5)
            speak(f"From: {article['source']}")
            time.sleep(0.5)
            speak(f"Summary: {summary}")
            time.sleep(1)  # Pause between articles

    return '\n'.join(formatted_output)

def main():
    # Initialize pygame mixer
    pygame.mixer.init()

    # Initialize summarizer
    #summarizer = pipeline("summarization")

    print("Welcome to the Stock News Assistant!")
    speak("Welcome to the Stock News Assistant! Say exit to quit.")
    time.sleep(1)

    while True:
        # Step 1: Get voice input
        spoken_text = get_voice_input()
        if not spoken_text:
            continue

        # Check for exit command
        if 'EXIT' in spoken_text:
            speak("Thank you for using the Stock News Assistant. Goodbye!")
            break

        # Step 2: Extract ticker
        ticker = extract_ticker(spoken_text)
        if not ticker:
            speak("Could not identify a ticker symbol. Please try again.")
            continue

        msg = f"Processing news for {ticker}..."
        print(msg)
        speak(msg)

        # Step 3: Fetch news
        news_data = get_news(ticker)
        if not news_data:
            speak("Could not fetch news data. Please try again.")
            continue

        # Step 4 & 5: Format and summarize news
        formatted_news = format_news(news_data, summarizer)
        print("\n" + formatted_news)

        speak("You can say another company name or say exit to quit.")
        time.sleep(1)

    # Clean up pygame
    pygame.mixer.quit()

if __name__ == "__main__":
    main()